In [1]:
#imports
import pandas as pd

In [2]:
circuits_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/circuits.csv')

constructor_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/constructors.csv')
constructor_results_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/constructor_results.csv')
constructor_standings_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/constructor_standings.csv')

drivers_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/drivers.csv')
drivers_standings_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/driver_standings.csv')

lap_times_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/lap_times.csv')

qualifying_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/qualifying.csv')

races_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/races.csv')
result_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/results.csv')

season_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/seasons.csv')

sprint_results_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/sprint_results.csv')

status_df = pd.read_csv('../input/formula-1-world-championship-1950-2020/status.csv')

## Lewis Hamilton Records
(Principios 2025 antes de comenzar la temporada del 2025)
### El piloto con más puntos actualmente

In [3]:
drivers_standings_df['year'] = drivers_standings_df['raceId'].map(races_df.set_index('raceId')['year'])

# Tomar solo el último registro de cada temporada para cada piloto (el acumulado final de la temporada)
last_standing_per_season = drivers_standings_df.sort_values(by=['year', 'raceId']).drop_duplicates(subset=['year', 'driverId'], keep='last')

# Sumar los puntos por piloto
total_points = last_standing_per_season.groupby('driverId')['points'].sum().reset_index()

# Unir con la tabla de pilotos para obtener nombres
drivers_df = drivers_df[['driverId', 'forename', 'surname']]
total_points = total_points.merge(drivers_df, on='driverId')

# Ordenar de mayor a menor
top_driver = total_points.sort_values(by='points', ascending=False).iloc[0]

# Mostrar el resultado
print(f" 🏁 El piloto con más puntos es {top_driver['forename']} {top_driver['surname']} con un total de {top_driver['points']} puntos. 🏁")

 🏁 El piloto con más puntos es Lewis Hamilton con un total de 4862.5 puntos. 🏁


### El piloto con más carreras ganadas

In [4]:
# Filtrar solo las victorias
wins_df = result_df[result_df['positionOrder'] == 1]

# Contar las victorias por piloto
wins_count = wins_df['driverId'].value_counts().reset_index()
wins_count.columns = ['driverId', 'wins']

# Unir con la tabla de pilotos para obtener nombres
drivers_df = drivers_df[['driverId', 'forename', 'surname']]
wins_count = wins_count.merge(drivers_df, on='driverId')

# Ordenar por número de victorias
top_winner = wins_count.sort_values(by='wins', ascending=False).iloc[0]

# Mostrar el resultado
print(f" 🚀 El piloto con más victorias es {top_winner['forename']} {top_winner['surname']} con {top_winner['wins']} carreras ganadas. 🚀")

 🚀 El piloto con más victorias es Lewis Hamilton con 105 carreras ganadas. 🚀


### El piloto con más podios

In [5]:
# Filtrar solo las posiciones de podio (1.º, 2.º o 3.º lugar)
podiums_df = result_df[result_df['positionOrder'].isin([1, 2, 3])]

# Contar la cantidad de podios por piloto
podium_count = podiums_df['driverId'].value_counts().reset_index()
podium_count.columns = ['driverId', 'podiums']

# Unir con la tabla de pilotos para obtener nombres
drivers_df = drivers_df[['driverId', 'forename', 'surname']]
podium_count = podium_count.merge(drivers_df, on='driverId')

# Ordenar por número de podios
top_podium_driver = podium_count.sort_values(by='podiums', ascending=False).iloc[0]

# Mostrar el resultado
print(f" 🔥 El piloto con más podios es {top_podium_driver['forename']} {top_podium_driver['surname']} con {top_podium_driver['podiums']} podios. 🔥")


 🔥 El piloto con más podios es Lewis Hamilton con 202 podios. 🔥


### El piloto con más pole positions
Pole positions = Salir primero en una carrera

In [6]:
pole_positions_df = qualifying_df[qualifying_df['position'] == 1]

# Contar cuántas pole positions tiene cada piloto
pole_count = pole_positions_df['driverId'].value_counts().reset_index()
pole_count.columns = ['driverId', 'pole_positions']

# Unir con la tabla de pilotos para obtener nombres
drivers_df = drivers_df[['driverId', 'forename', 'surname']]
pole_count = pole_count.merge(drivers_df, on='driverId')

# Ordenar por número de poles
top_pole_driver = pole_count.sort_values(by='pole_positions', ascending=False).iloc[0]

# Mostrar el resultado
print(f"🏆 El piloto con más pole positions es {top_pole_driver['forename']} {top_pole_driver['surname']} con {top_pole_driver['pole_positions']} pole positions. 🏆")

🏆 El piloto con más pole positions es Lewis Hamilton con 107 pole positions. 🏆


### El piloto con más carreras lideradas 
(al menos una vuelta liderada en carrera)

In [7]:
# Filtrar vueltas donde un piloto estuvo en P1
laps_led = lap_times_df[lap_times_df['position'] == 1]

# Contar en cuántas carreras diferentes lideró cada piloto (cada raceId único cuenta una vez)
races_led_count = laps_led.groupby('driverId')['raceId'].nunique().reset_index()
races_led_count.columns = ['driverId', 'races_led']

# Unir con la tabla de pilotos para obtener nombres
races_led_count = races_led_count.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
races_led_count = races_led_count.sort_values(by='races_led', ascending=False)

# Mostrar el piloto con más carreras lideradas
top_driver = races_led_count.iloc[0]
print(f"🔝 El piloto con más carreras lideradas es {top_driver['forename']} {top_driver['surname']} con {top_driver['races_led']} carreras lideradas. 🔝")


🔝 El piloto con más carreras lideradas es Lewis Hamilton con 190 carreras lideradas. 🔝


### El piloto con más vueltas lideradas
Total de vueltas en P1

In [8]:
# Contar cuántas vueltas lideró cada piloto
laps_led_count = laps_led['driverId'].value_counts().reset_index()
laps_led_count.columns = ['driverId', 'laps_led']

# Unir con la tabla de pilotos
laps_led_count = laps_led_count.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
laps_led_count = laps_led_count.sort_values(by='laps_led', ascending=False)

# Mostrar el piloto con más vueltas lideradas
top_driver = laps_led_count.iloc[0]
print(f"🎯 El piloto con más vueltas lideradas es {top_driver['forename']} {top_driver['surname']} con {top_driver['laps_led']} vueltas lideradas. 🎯")

🎯 El piloto con más vueltas lideradas es Lewis Hamilton con 5487 vueltas lideradas. 🎯


### El piloto con más campeonatos mundiales

In [9]:
# Obtener la última carrera de cada temporada
last_race_per_season = races_df.groupby('year')['raceId'].max().reset_index()

# Filtrar la tabla de standings para obtener SOLO los campeones de cada temporada
champions = drivers_standings_df[
    (drivers_standings_df['position'] == 1) & 
    (drivers_standings_df['raceId'].isin(last_race_per_season['raceId']))
]

# Contar cuántos títulos tiene cada piloto
championships_count = champions['driverId'].value_counts().reset_index()
championships_count.columns = ['driverId', 'titles']

# Unir con la tabla de pilotos para obtener nombres
championships_count = championships_count.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
championships_count = championships_count.sort_values(by='titles', ascending=False)

# Obtener el máximo número de títulos
max_titles = championships_count['titles'].max()

# Filtrar todos los pilotos que tengan ese número de títulos
top_drivers = championships_count[championships_count['titles'] == max_titles]

# Mostrar los pilotos con más títulos
for _, row in top_drivers.iterrows():
    print(f"🏁 {row['forename']} {row['surname']} tiene {row['titles']} títulos. 🏁")


🏁 Lewis Hamilton tiene 7 títulos. 🏁
🏁 Michael Schumacher tiene 7 títulos. 🏁


### El piloto con mas victorias en diferentes temporadas

In [10]:
# Filtrar las carreras ganadas (positionOrder == 1 indica el ganador de la carrera)
winners = result_df[result_df['positionOrder'] == 1]

# Unir con la tabla de carreras para obtener el año de cada carrera ganada
winners = winners.merge(races_df[['raceId', 'year']], on='raceId')

# Contar en cuántas temporadas diferentes ganó al menos una carrera cada piloto
seasons_won = winners.groupby('driverId')['year'].nunique().reset_index()
seasons_won.columns = ['driverId', 'seasons_won']

# Unir con la tabla de pilotos para obtener nombres
seasons_won = seasons_won.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
seasons_won = seasons_won.sort_values(by='seasons_won', ascending=False)

# Obtener el número máximo de temporadas ganadas
max_seasons = seasons_won['seasons_won'].max()

# Filtrar todos los pilotos que tengan ese número de temporadas ganadas
top_drivers = seasons_won[seasons_won['seasons_won'] == max_seasons]

# Mostrar los pilotos con más temporadas ganando carreras
for _, row in top_drivers.iterrows():
    print(f"🪄 {row['forename']} {row['surname']} ha ganado en {row['seasons_won']} temporadas diferentes. 🪄")


🪄 Lewis Hamilton ha ganado en 16 temporadas diferentes. 🪄


### El piloto con más victorias en x circuito 

In [11]:
# Filtrar las carreras ganadas (positionOrder == 1 indica el ganador de la carrera)
winners = result_df[result_df['positionOrder'] == 1]

# Unir con la tabla de carreras para obtener el circuito de cada carrera ganada
winners = winners.merge(races_df[['raceId', 'circuitId']], on='raceId')

# Contar cuántas veces cada piloto ha ganado en cada circuito
circuit_wins = winners.groupby(['driverId', 'circuitId']).size().reset_index(name='wins')

# Encontrar el máximo número de victorias en un solo circuito
max_wins = circuit_wins['wins'].max()

# Filtrar todos los pilotos que tengan ese número de victorias en algún circuito
top_circuit_winners = circuit_wins[circuit_wins['wins'] == max_wins]

# Unir con la tabla de pilotos para obtener nombres
top_circuit_winners = top_circuit_winners.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Unir con la tabla de circuitos para obtener el nombre del circuito
top_circuit_winners = top_circuit_winners.merge(circuits_df[['circuitId', 'name']], on='circuitId')

# Mostrar los pilotos con más victorias en un mismo circuito
for _, row in top_circuit_winners.iterrows():
    print(f"🥇 {row['forename']} {row['surname']} ha ganado {row['wins']} veces en {row['name']}. 🥇")


🥇 Lewis Hamilton ha ganado 9 veces en Silverstone Circuit. 🥇


### El piloto con más victorias como rookie

In [12]:
# Obtener la primera temporada de cada piloto (año de su primera carrera)
first_season = races_df[['raceId', 'year']].merge(result_df[['raceId', 'driverId']], on='raceId')
first_season = first_season.groupby('driverId')['year'].min().reset_index()
first_season.columns = ['driverId', 'rookie_year']

# Filtrar las victorias (positionOrder == 1) y unir con la temporada de debut
rookie_wins = result_df[result_df['positionOrder'] == 1].merge(races_df[['raceId', 'year']], on='raceId')
rookie_wins = rookie_wins.merge(first_season, on='driverId')

# Filtrar solo las victorias en la temporada de debut
rookie_wins = rookie_wins[rookie_wins['year'] == rookie_wins['rookie_year']]

# Contar las victorias de cada piloto en su año de debut
rookie_win_count = rookie_wins['driverId'].value_counts().reset_index()
rookie_win_count.columns = ['driverId', 'rookie_wins']

# Unir con la tabla de pilotos para obtener nombres
rookie_win_count = rookie_win_count.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
rookie_win_count = rookie_win_count.sort_values(by='rookie_wins', ascending=False)

# Obtener el número máximo de victorias de un rookie
max_rookie_wins = rookie_win_count['rookie_wins'].max()

# Filtrar todos los pilotos que lograron esa cantidad de victorias
top_rookies = rookie_win_count[rookie_win_count['rookie_wins'] == max_rookie_wins]

# Mostrar los rookies con más victorias
for _, row in top_rookies.iterrows():
    print(f"🎖️ {row['forename']} {row['surname']} ganó {row['rookie_wins']} carreras en su temporada de debut ({row['driverId']}). 🎖️")


🎖️ Lewis Hamilton ganó 4 carreras en su temporada de debut (1). 🎖️
🎖️ Jacques Villeneuve ganó 4 carreras en su temporada de debut (35). 🎖️


### El piloto con más pole position como rookie

In [13]:
# Obtener la primera temporada de cada piloto (año de su primera carrera)
first_season = races_df[['raceId', 'year']].merge(result_df[['raceId', 'driverId']], on='raceId')
first_season = first_season.groupby('driverId')['year'].min().reset_index()
first_season.columns = ['driverId', 'rookie_year']

# Filtrar las poles (posición 1 en clasificación)
rookie_poles = qualifying_df[qualifying_df['position'] == 1]

# Unir con la tabla de carreras para obtener el año de la pole
rookie_poles = rookie_poles.merge(races_df[['raceId', 'year']], on='raceId')

# Unir con la primera temporada de cada piloto
rookie_poles = rookie_poles.merge(first_season, on='driverId')

# Filtrar solo las poles en la temporada de debut
rookie_poles = rookie_poles[rookie_poles['year'] == rookie_poles['rookie_year']]

# Contar las poles de cada piloto en su año de debut
rookie_pole_count = rookie_poles['driverId'].value_counts().reset_index()
rookie_pole_count.columns = ['driverId', 'rookie_poles']

# Unir con la tabla de pilotos para obtener nombres
rookie_pole_count = rookie_pole_count.merge(drivers_df[['driverId', 'forename', 'surname']], on='driverId')

# Ordenar de mayor a menor
rookie_pole_count = rookie_pole_count.sort_values(by='rookie_poles', ascending=False)

# Obtener el número máximo de poles de un rookie
max_rookie_poles = rookie_pole_count['rookie_poles'].max()

# Filtrar todos los pilotos que lograron esa cantidad de poles
top_rookies = rookie_pole_count[rookie_pole_count['rookie_poles'] == max_rookie_poles]

# Mostrar los rookies con más pole positions
for _, row in top_rookies.iterrows():
    print(f"🚀 {row['forename']} {row['surname']} logró {row['rookie_poles']} pole positions en su temporada de debut. 🚀")


🚀 Lewis Hamilton logró 5 pole positions en su temporada de debut. 🚀


### El piloto con más premios con un mismo constructor

In [14]:
# Crear una nueva columna con el nombre completo del piloto
drivers_df['fullName'] = drivers_df['forename'] + ' ' + drivers_df['surname']

# 1. Más Grandes Premios con un mismo constructor
# Agrupar por piloto y constructor en result_df y contar las carreras
driver_constructor_grands_prix = result_df.groupby(['driverId', 'constructorId'])['raceId'].nunique().reset_index()

# Unir con la tabla de drivers para obtener el nombre completo del piloto
driver_constructor_grands_prix = driver_constructor_grands_prix.merge(drivers_df[['driverId', 'fullName']], on='driverId', how='left')

# Unir con la tabla de constructores para obtener el nombre del constructor
driver_constructor_grands_prix = driver_constructor_grands_prix.merge(constructor_df[['constructorId', 'name']], on='constructorId', how='left')

# Encontramos el piloto con más Grandes Premios con un mismo constructor
max_grands_prix_constructor = driver_constructor_grands_prix.loc[driver_constructor_grands_prix['raceId'].idxmax()]

# Imprimir el resultado con los nombres de piloto y constructor
print(f"🏎️ Más Grandes Premios con un mismo constructor: {max_grands_prix_constructor['fullName']} con {max_grands_prix_constructor['raceId']} carreras en {max_grands_prix_constructor['name']}. 🏎️")


🏎️ Más Grandes Premios con un mismo constructor: Lewis Hamilton con 246 carreras en Mercedes. 🏎️


### El piloto con más carreras consecutivas finalizadas con puntos

In [15]:
# Filtrar carreras donde el piloto obtuvo puntos (por ejemplo, posición 1-10)
result_with_points = result_df[result_df['positionOrder'] <= 10]

# Ordenar por piloto y raceId para asegurar que las carreras están en orden cronológico
result_with_points = result_with_points.sort_values(by=['driverId', 'raceId'])

# Crear una columna de "carrera consecutiva" para cada piloto
result_with_points['consecutive'] = result_with_points.groupby('driverId')['raceId'].diff().ne(1).cumsum()

# Contamos las carreras consecutivas con puntos para cada piloto
consecutive_points = result_with_points.groupby(['driverId', 'consecutive']).size().reset_index(name='consecutive_points')

# Encontrar el piloto con más carreras consecutivas con puntos
max_consecutive_points = consecutive_points.loc[consecutive_points['consecutive_points'].idxmax()]

# Obtener el nombre del piloto
driver_name = drivers_df.loc[drivers_df['driverId'] == max_consecutive_points['driverId'], 'fullName'].values[0]

# Imprimir el resultado
print(f"🏁 El piloto con más carreras consecutivas finalizadas con puntos es {driver_name} con {max_consecutive_points['consecutive_points']} carreras consecutivas. 🏁")


🏁 El piloto con más carreras consecutivas finalizadas con puntos es Lewis Hamilton con 48 carreras consecutivas. 🏁


### El piloto con más victorias en diferentes circuitos 

In [16]:
# Filtrar las victorias (posición 1)
victories = result_df[result_df['positionOrder'] == 1]

# Unir con races_df para obtener el circuito (circuitId) de cada carrera
victories = victories.merge(races_df[['raceId', 'circuitId']], on='raceId', how='left')

# Agrupar por piloto y circuito para contar las victorias en diferentes circuitos
victories_by_circuit = victories.groupby(['driverId', 'circuitId']).size().reset_index(name='victories')

# Contar las victorias por piloto en circuitos diferentes (sin contar repeticiones en el mismo circuito)
driver_victories_in_circuits = victories_by_circuit.groupby('driverId')['circuitId'].nunique().reset_index(name='unique_victories')

# Encontrar el piloto con más victorias en circuitos diferentes
max_victories = driver_victories_in_circuits.loc[driver_victories_in_circuits['unique_victories'].idxmax()]

# Obtener el nombre del piloto
driver_name = drivers_df.loc[drivers_df['driverId'] == max_victories['driverId'], 'forename'].values[0] + ' ' + drivers_df.loc[drivers_df['driverId'] == max_victories['driverId'], 'surname'].values[0]

# Imprimir el resultado
print(f"🚀 El piloto con más victorias en diferentes circuitos es {driver_name} con {max_victories['unique_victories']} victorias. 🚀")


🚀 El piloto con más victorias en diferentes circuitos es Lewis Hamilton con 31 victorias. 🚀


### El piloto con más podios en diferentes circuitos

In [17]:
# Filtrar los resultados de los podios (posiciones 1, 2, 3)
podiums = result_df[result_df['positionOrder'].isin([1, 2, 3])]

# Unir con races_df para obtener el circuito (circuitId) de cada carrera
podiums = podiums.merge(races_df[['raceId', 'circuitId']], on='raceId', how='left')

# Agrupar por piloto y circuito para contar los podios en diferentes circuitos
podiums_by_circuit = podiums.groupby(['driverId', 'circuitId']).size().reset_index(name='podiums')

# Contar los podios por piloto en circuitos diferentes (sin contar repeticiones en el mismo circuito)
driver_podiums_in_circuits = podiums_by_circuit.groupby('driverId')['circuitId'].nunique().reset_index(name='unique_podiums')

# Encontrar el piloto con más podios en circuitos diferentes
max_podiums = driver_podiums_in_circuits.loc[driver_podiums_in_circuits['unique_podiums'].idxmax()]

# Obtener el nombre del piloto
driver_name = drivers_df.loc[drivers_df['driverId'] == max_podiums['driverId'], 'forename'].values[0] + ' ' + drivers_df.loc[drivers_df['driverId'] == max_podiums['driverId'], 'surname'].values[0]

# Imprimir el resultado
print(f"💥 El piloto con más podios en diferentes circuitos es {driver_name} con {max_podiums['unique_podiums']} podios. 💥")


💥 El piloto con más podios en diferentes circuitos es Lewis Hamilton con 36 podios. 💥


## Resumen
Hamilton es el piloto actual con más records. Tras 14 años consecutivos con Mercedes, abre una nueva era con Ferrari. ¿Podrá seguir batiendo records?
Lewis Hamilton es el piloto con más record en el mundo de la formula 1. A día de hoy es considerado uno de los mejores pilotos. Comenzo en la formula 1 con McLaren en 2007 con 22 años. En 2013 firmo un contrato con Mercedes y estarían juntos hasta 2024 que decidio cumplir su mayor sueño. Poder correr con Ferrari.
A lo largo de su carrera cumplió todos los records que acabamos de ver. Seguramente tenga más record, pero necesitaríamos más información.

1. 🏁 El piloto con más puntos es Lewis Hamilton con un total de 4862.5 puntos. 🏁
2. 🚀 El piloto con más victorias es Lewis Hamilton con 105 carreras ganadas. 🚀
3. 🔥 El piloto con más podios es Lewis Hamilton con 202 podios. 🔥
4. 🏆 El piloto con más pole positions es Lewis Hamilton con 107 pole positions. 🏆
6. 🔝 El piloto con más carreras lideradas es Lewis Hamilton con 190 carreras lideradas. 🔝
7. 🎯 El piloto con más vueltas lideradas es Lewis Hamilton con 5487 vueltas lideradas. 🎯
8. 🏁 Lewis Hamilton y Michael Schumacher tienen ambos 7 títulos mundiales. ¿Será capaz Hamilton de batir el record en su nueva etapa con Ferrari? 🏁
9. 🪄 Lewis Hamilton ha ganado en 16 temporadas diferentes. 🪄
10. 🥇 Lewis Hamilton ha ganado 9 veces en Silverstone Circuit. 🥇
11. 🎖️ Lewis Hamilton y Jacques Villeneuve comparten el record de ganar 4 carreras en su temporada de debut 🎖️
12. 🚀 Lewis Hamilton logró 5 pole positions en su temporada de debut. 🚀
13. 🏎️ Más Grandes Premios con un mismo constructor: Lewis Hamilton con 246 carreras en Mercedes. 🏎️
14. 🏁 El piloto con más carreras consecutivas finalizadas con puntos es Lewis Hamilton con 48 carreras consecutivas. 🏁
15. 🚀 El piloto con más victorias en diferentes circuitos es Lewis Hamilton con 31 victorias. 🚀
16. 💥 El piloto con más podios en diferentes circuitos es Lewis Hamilton con 36 podios. 💥
